#### 1. generating example data 

In [220]:
import pandas as pd
import numpy as np
import random as rd

#df with 10 ASVs x 10 samples populated by random integers between 0 and 99
df = pd.DataFrame(np.random.randint(0,100,size=(10, 10)), 
                  columns=['s1','s2','s3','s4','s5','s6','s7','s8','s9','s10'], 
                  index = ["{}_{}".format('ASV', i) for i in range(1, 11)])

df

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10
ASV_1,0,86,16,19,9,92,51,10,68,23
ASV_2,14,63,21,46,3,56,88,80,46,54
ASV_3,79,71,14,77,15,25,53,84,58,85
ASV_4,29,44,37,22,91,54,76,12,59,26
ASV_5,76,71,87,39,43,76,38,91,69,98
ASV_6,33,43,26,56,69,73,52,89,27,43
ASV_7,1,82,94,96,26,84,30,64,22,98
ASV_8,52,3,70,12,39,83,48,61,70,13
ASV_9,36,23,22,66,92,95,53,9,41,57
ASV_10,88,93,76,82,11,36,16,30,84,57


#### 2. defining SRS function 

In [233]:
def SRS(data, Cmin, set_seed = True, seed = 1):
    
    #set the seed that allows reproducible results
    if set_seed is True:
        np.random.seed(seed)
    
    #check whether samples will have to be discarded due to low library size (sequencing depth)
    if Cmin > min(data.sum(axis = 0)):
        max_lib_size = max(data.sum(axis = 0))
        if Cmin > max_lib_size:
            print("Please select a Cmin <= ", max_lib_size)
            raise ValueError("All samples discarded due to low number of counts.")
        else:
            discarded = data.columns.values[data.sum() < Cmin]
            print("Sample(s) discarded due to low number of counts (number of counts < Cmin): ", end='')
            print(*discarded, sep=", ")
            data = data[data.columns[data.sum() >= Cmin]]
            samples = data.columns
            SRS(data, Cmin, set_seed, seed)
            
    #proceed with samples with library size >= Cmin
    else:
        
        #throw error in case of invalid Cmin
        if Cmin < 0:
            raise ValueError("Cmin < 0. Please select Cmin >= 0.")
        if float(Cmin).is_integer() is False:
            raise ValueError("SRS accepts only integers for Cmin.")
        
        #start normalization
        else:
            #scaling
            data_scaled = data.div(data.sum(axis = 0) / Cmin, axis = 1)
            #picking the integer part of the scaled data
            data_scaled_floor = np.floor(data_scaled)
            #picking the fractional part of the scaled data
            data_scaled_frac = data_scaled - data_scaled_floor
            #n of counts missing for each sample in the integer
            #part of the scaled data in order to reach Cmin
            counts_missing = data_scaled_frac.sum(axis = 0)
            data_normalized = data_scaled_floor
            
            #start distribution of missing counts per sample
            for sample in data.columns:
                counts_missing_s = int(counts_missing[sample])
                data_scaled_frac_s = data_scaled_frac[sample]
                data_scaled_frac_s_rank = data_scaled_frac_s.rank(method = 'min').sort_values(ascending = False)
                data_scaled_floor_s = data_scaled_floor[sample]
                while counts_missing_s > 0:
                    for asv in data_scaled_frac_s_rank.index:
                        if counts_missing_s == 0:
                            break
                        rank_current_asv = data_scaled_frac_s_rank.loc[asv]
                        n_asvs_sharing_frac_rank = data_scaled_frac_s_rank.value_counts()[rank_current_asv]
                        
                        #distribute counts based on the frac rank (descending)
                        if n_asvs_sharing_frac_rank <= counts_missing_s:
                            data_normalized.loc[asv, sample] = data_normalized.loc[asv, sample] + 1
                            counts_missing_s -= 1
                            data_scaled_frac_s_rank = data_scaled_frac_s_rank.drop(asv)
                        
                        #if there are asvs sharing the same frac rank and there are less
                        #missing counts than the number of asvs sharing the same frac rank
                        else:
                            asvs_sharing_frac_rank = data_scaled_frac_s_rank.head(n_asvs_sharing_frac_rank).index
                            data_scaled_floor_s_asfr = data_scaled_floor_s.filter(asvs_sharing_frac_rank)
                            data_scaled_floor_s_asfr_rank = data_scaled_floor_s_asfr.rank(method = 'min').sort_values(ascending = False)
                            for asv in data_scaled_floor_s_asfr_rank.index:
                                if counts_missing_s == 0:
                                    break
                                rank_current_asv = data_scaled_floor_s_asfr_rank.loc[asv]
                                n_asvs_sharing_floor_rank = data_scaled_floor_s_asfr_rank.value_counts()[rank_current_asv]
                                
                                #distribute counts based on the int rank (descending)
                                if n_asvs_sharing_floor_rank <= counts_missing_s:
                                    data_normalized.loc[asv, sample] = data_normalized.loc[asv, sample] + 1
                                    counts_missing_s -= 1
                                    data_scaled_floor_s_asfr_rank = data_scaled_floor_s_asfr_rank.drop(asv)
                                
                                #if these asvs also share the same int rank, then
                                #distribute missing counts randomly (without replacement)
                                else:
                                    while counts_missing_s > 0:
                                        random_asv = rd.choice(data_scaled_floor_s_asfr_rank.index)
                                        data_normalized.loc[random_asv, sample] = data_normalized.loc[random_asv, sample] + 1
                                        counts_missing_s -= 1
                                        data_scaled_floor_s_asfr_rank = data_scaled_floor_s_asfr_rank.drop(random_asv)
    
    #return df with SRS-normalized data (counts)
    return(data_normalized.astype(int))

#### 3. running SRS function 

In [235]:
SRS(df, Cmin = 120)

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10
ASV_1,0,18,4,4,2,16,12,2,15,5
ASV_2,4,13,5,11,1,10,21,18,10,12
ASV_3,23,14,4,18,4,4,13,19,13,19
ASV_4,9,9,9,5,27,10,18,3,13,6
ASV_5,22,15,22,9,13,14,9,20,15,21
ASV_6,10,9,7,13,21,13,12,20,6,9
ASV_7,0,17,24,22,8,15,7,14,5,21
ASV_8,15,0,18,3,12,15,11,14,15,3
ASV_9,11,5,6,15,28,17,13,2,9,12
ASV_10,26,19,20,19,3,6,4,7,19,12


check https://cran.r-project.org/web/packages/SRS/SRS.pdf (page 2) for details on SRS function params